In [25]:
def permute(bitlist, permute_list):
    max_index = max(permute_list)
    if max_index >= len(bitlist):
        raise ValueError("Bad permutation index")
    outlist = [bitlist[index] for index in permute_list]
    return outlist

def substitute(expanded_half_block, s_boxes, sbox_index):
    output = []
    if len(expanded_half_block) != 48:
        raise ValueError("Expanded half block length must be 48 bits")
    
    segments = [expanded_half_block[x * 6:(x + 1) * 6] for x in range(8)]
    for sindex, segment in enumerate(segments):
        if len(segment) != 6:
            raise ValueError("Segment length must be 6 bits")
        
        row = int(''.join(map(str, segment[:2])), 2)
        column = int(''.join(map(str, segment[4:])), 2)
        output.extend([int(x) for x in format(s_boxes[sbox_index][row][column], '04b')])
    return output

In [26]:
def generate_round_keys(encryption_key, key_permutation_1, key_permutation_2, shifts_for_round_key_gen):
    round_keys = []
    key = permute(encryption_key, key_permutation_1)
    for round_count in range(16):
        LKey = key[:28]
        RKey = key[28:]
        shift = shifts_for_round_key_gen[round_count]
        LKey = LKey[shift:] + LKey[:shift]
        RKey = RKey[shift:] + RKey[:shift]
        key = LKey + RKey
        round_key = permute(key, key_permutation_2)
        round_keys.append(round_key)
    return round_keys

In [27]:
def single_round_DES(right_half_block, round_key, expansion_permutation, s_boxes):
    right_half_block_bits = [int(x) for x in format(right_half_block, '064b')]
    expanded_block = permute(right_half_block_bits, expansion_permutation)
    mixed_block = [expanded_block[i] ^ round_key[i] for i in range(len(expanded_block))]
    substituted_block = substitute(mixed_block, s_boxes, 0)
    permuted_block = permute(substituted_block, [15, 6, 19, 20, 28, 11, 27, 16,
                                                  0, 14, 22, 25, 4, 17, 30, 9,
                                                  1, 7, 23, 13, 31, 26, 2, 8,
                                                  18, 12, 29, 5, 21, 10, 3, 24])
    return permuted_block

In [28]:
def des_encrypt(message, key, key_permutation_1, key_permutation_2, shifts_for_round_key_gen,
                expansion_permutation, s_boxes):
    round_keys = generate_round_keys(key, key_permutation_1, key_permutation_2, shifts_for_round_key_gen)
    print("Round Keys:")
    for rk in round_keys:
        print_binary(rk)
    ciphertext = []
    for block in message:
        LE = block >> 64
        RE = block & 0xFFFFFFFFFFFFFFFF
        for round_key in round_keys:
            newRE = LE ^ int(''.join(map(str, single_round_DES(RE, round_key, expansion_permutation, s_boxes))), 2)
            LE = RE
            RE = newRE
        ciphertext.append((RE << 64) | LE)
    return ciphertext

In [29]:
def print_binary(bits):
    for block in bits:
        print(''.join(str((block >> i) & 1) for i in range(63, -1, -1)))

def string_to_bits(string):
    return [int(x) for x in ''.join(format(ord(c), '08b') for c in string)]

def bits_to_string(bits):
    bit_str = ''.join(str(b) for b in bits)
    return ''.join(chr(int(bit_str[i:i+8], 2)) for i in range(0, len(bit_str), 8))

In [30]:
# Define the expansion permutation and S-boxes
expansion_permutation = [31, 0, 1, 2, 3, 4,
                         3, 4, 5, 6, 7, 8,
                         7, 8, 9, 10, 11, 12,
                         11, 12, 13, 14, 15, 16,
                         15, 16, 17, 18, 19, 20,
                         19, 20, 21, 22, 23, 24,
                         23, 24, 25, 26, 27, 28,
                         27, 28, 29, 30, 31, 0]

In [31]:
s_boxes = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],

    [        
              [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
              [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
              [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
              [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],

[             [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
              [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
              [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
              [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
              ],

[             [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
              [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
              [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
              [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
              ],

[             [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
              [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
              [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
              [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
              ],

 [            [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
              [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
              [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
              [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
              ],

 [            [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
              [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
              [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
              [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
              ],

[             [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
              [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
              [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
              [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
              ]
]

In [32]:
# Define the remaining constants needed for DES
key_permutation_1 = [56, 48, 40, 32, 24, 16, 8,
                     0, 57, 49, 41, 33, 25, 17,
                     9, 1, 58, 50, 42, 34, 26,
                     18, 10, 2, 59, 51, 43, 35,
                     62, 54, 46, 38, 30, 22, 14,
                     6, 61, 53, 45, 37, 29, 21,
                     13, 5, 60, 52, 44, 36, 28,
                     20, 12, 4, 27, 19, 11, 3]

key_permutation_2 = [13, 16, 10, 23, 0, 4,
                     2, 27, 14, 5, 20, 9,
                     22, 18, 11, 3, 25, 7,
                     15, 6, 26, 19, 12, 1,
                     40, 51, 30, 36, 46, 54,
                     29, 39, 50, 44, 32, 47,
                     43, 48, 38, 55, 33, 52,
                     45, 41, 49, 35, 28, 31]

In [33]:
message = [int(''.join(map(str, block)), 2) for block in [string_to_bits("Hello, World!")]]
key = string_to_bits("This is a key!")
encrypted_message = des_encrypt(message, key, key_permutation_1, key_permutation_2, shifts_for_round_key_gen,
                                expansion_permutation, s_boxes)
print("Encrypted message:")
print_binary(encrypted_message)

Round Keys:
...
Encrypted message:
0000000000000000000000000100100010000101110010100111111010000001


In [34]:
import pprint

def create_presentation():
    presentation = {
        "Title": "DES (Data Encryption Standard) Implementation",
        "Description": "This is an implementation of the DES algorithm, a widely used encryption technique for securing data.",
        "Key Features": [
            "Converts plaintext to binary representation",
            "Generates 16 round keys from the secret key",
            "Performs 16 rounds of operations on each 64-bit block",
            "Includes helper functions for permutation, substitution, and key generation",
            "Produces an encrypted ciphertext as output"
        ],
        "Example Usage": {
            "Plaintext": "Hello, World!",
            "Secret Key": "This is a key!",
            "Encrypted Message": encrypted_message
        }
    }
    
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(presentation)

# Assuming you have the `encrypted_message` variable from the previous code
create_presentation()

{   'Description': 'This is an implementation of the DES algorithm, a widely '
                   'used encryption technique for securing data.',
    'Example Usage': {   'Encrypted Message': [   58653958750823321791302443713071578753],
                         'Plaintext': 'Hello, World!',
                         'Secret Key': 'This is a key!'},
    'Key Features': [   'Converts plaintext to binary representation',
                        'Generates 16 round keys from the secret key',
                        'Performs 16 rounds of operations on each 64-bit block',
                        'Includes helper functions for permutation, '
                        'substitution, and key generation',
                        'Produces an encrypted ciphertext as output'],
    'Title': 'DES (Data Encryption Standard) Implementation'}
